<a href="https://colab.research.google.com/github/janepu47/229352/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Lab04_Naive_Bayes_Grid_and_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [2]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [3]:
from sklearn.naive_bayes import MultinomialNB

nd = MultinomialNB(alpha = 0.1)


### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
pipeline = Pipeline([('count', CountVectorizer(stop_words = 'english')), ('nb', MultinomialNB())])

Parameter = {'nb__alpha' : uniform(loc=0.1, scale = 10)}

clf = RandomizedSearchCV(pipeline, Parameter, cv = 3, n_iter = 10)
clf.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('count',
                                              CountVectorizer(stop_words='english')),
                                             ('nb', MultinomialNB())]),
                   param_distributions={'nb__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ad5da54e610>})

#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [5]:
# กำหนดค่าที่ต้องการทดสอบ
parameters = {
    'nb__alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # พารามิเตอร์ของ Naive Bayes
    'count__max_features': [1000, 2000, 5000]  # พารามิเตอร์ของ CountVectorizer (optional)
}


# สร้าง Grid Search ด้วย 5-fold CV
grid_search = GridSearchCV(pipeline, parameters, cv=5)

# ฝึกโมเดล (ยกคอมเมนต์เมื่อมีข้อมูลจริง)
grid_search.fit(Xtrain, ytrain)

# แสดงผลลัพธ์
print("ค่า alpha ที่ดีที่สุด:", grid_search.best_params_)
print("คะแนนที่ดีที่สุด:", grid_search.best_score_)

ค่า alpha ที่ดีที่สุด: {'count__max_features': 5000, 'nb__alpha': 0.1}
คะแนนที่ดีที่สุด: 0.7656666666666666


#####For the best value of alpha, compute the f1_macro score on the test set

In [6]:
from sklearn.metrics import f1_score
# Create pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

# Parameters to tune
parameters = {
    'clf__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0]
}

# Perform grid search with 5-fold CV
grid_search = GridSearchCV(text_clf, parameters, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(Xtrain, ytrain)

# Get best alpha and model
best_alpha = grid_search.best_params_['clf__alpha']
best_model = grid_search.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(Xtest)
f1_macro = f1_score(ytest, y_pred, average='macro')

print(f"Best alpha value: {best_alpha}")
print(f"F1-macro score on test set: {f1_macro:.4f}")

Best alpha value: 0.1
F1-macro score on test set: 0.7188


#####Repeat Exercise 1 and 2 for random search 5-fold cross validation across different values of alpha. Compute the f1_macro score on the test set.

In [7]:
from scipy.stats import uniform, randint

# กำหนดการกระจายตัวของพารามิเตอร์ (สุ่มจากช่วงต่อเนื่อง/ไม่ต่อเนื่อง)
param_dist = {
    'clf__alpha': uniform(0.01, 10),  # สุ่มค่า alpha จาก 0.01 ถึง 10.01
    'vect__max_features': randint(500, 10000)  # สุ่มจำนวน features จาก 500 ถึง 10000
}

# สร้าง RandomizedSearchCV
random_search = RandomizedSearchCV(
    text_clf,  # Pipeline เดิม
    param_distributions=param_dist,
    n_iter=20,  # จำนวนการสุ่มทดสอบ (ปรับตามความต้องการ)
    cv=5,       # 5-fold CV
    scoring='f1_macro',
    random_state=42,
    n_jobs=-1
)

# ฝึกโมเดล
random_search.fit(Xtrain, ytrain)

# แสดงผลลัพธ์
print("ค่า alpha ที่ดีที่สุดจาก Random Search:", random_search.best_params_['clf__alpha'])
print("คะแนนที่ดีที่สุด (CV):", random_search.best_score_)

# ประเมินบน test set
y_pred_random = random_search.predict(Xtest)
f1_random = f1_score(ytest, y_pred_random, average='macro')
print(f"F1-macro score บน test set (Random Search): {f1_random:.4f}")

# เปรียบเทียบกับ Grid Search
if f1_random > f1_macro:  # f1_macro คือคะแนนจาก Grid Search ที่คุณได้ก่อนหน้า
    print("Random Search ให้ผลลัพธ์ดีกว่า Grid Search")
else:
    print("Grid Search ให้ผลลัพธ์ดีกว่าหรือเท่ากัน")

ค่า alpha ที่ดีที่สุดจาก Random Search: 2.0067378215835974
คะแนนที่ดีที่สุด (CV): 0.7561119245510171
F1-macro score บน test set (Random Search): 0.6826
Grid Search ให้ผลลัพธ์ดีกว่าหรือเท่ากัน
